<a href="https://colab.research.google.com/github/mnshcodie/IIScEx_2021/blob/main/Copy_of_M3_AST_27_Convolutional_Neural_Networks_for_Sentence_Classification_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Programme in Deep Learning (Foundations and Applications)
## A Program by IISc and TalentSprint
### Assignment 27: Natural Language Processing - II (Convolutional Neural Networks for Sentence Classification)

### Learning Objectives:

At the end of the experiment, you will be able to:
 
*  represent words in the sentences using pretrained word embeddings
*  generate vector representation of words in the sentence using Glove
*  build a convolutional Neural Network for Sentence Classification


### Dataset

#### Description


The SICK(Sentences Involving Compositional Knowledge) dataset contains 9840 English sentence pairs:

- two sentences, SentenceA and SentenceB
- their relation to each other from SentenceA to SentenceB and from SentenceB to SentenceA (entailment, contradiction, neutral)

For more details about the dataset refer to the following [link](https://zenodo.org/record/2787612#.YbA5pCbhU8o)

### Problem Statement

The aim of this assignment is to study the use of a Siamese neural network for solving the textual
entailment problem. We will use the CNN based
architecture for sentence representation.

Every instance of this dataset consists of two sentences (A and B) and the information whether the sentence (A/B)
contradicts/neutral/entails the sentence (B/A). We treat this as a seven-category classification problem
and design a Siamese neural network for solving this classification problem. Note that the input to
the Siamese network will be two sentences and the output could be a 7-dimensional vector for the
7 classes.

### Setup Steps:

In [ ]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "" #@param {type:"string"}

In [ ]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "" #@param {type:"string"}

In [ ]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython
import warnings
warnings.filterwarnings("ignore")

ipython = get_ipython()
  
notebook= "M3_AST_27_Convolutional_Neural_Networks_for_Sentence_Classification_C" #name of the notebook

def setup():
#  ipython.magic("sx pip3 install torch") 
    ipython.magic("sx wget -qq https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/glove.6B.zip")
    ipython.magic("sx unzip glove.6B.zip")
    ipython.magic("sx wget https://cdn.iisc.talentsprint.com/DLFA/Experiment_related_data/sententence_data.csv")


    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    ipython.magic("notebook -e "+ notebook + ".ipynb")
    
    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:        
        print(r["err"])
        return None        
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None
    
    elif getAnswer1() and getAnswer2() and getComplexity() and getAdditional() and getConcepts() and getComments() and getMentorSupport():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional, 
              "concepts" : Concepts, "record_id" : submission_id, 
              "answer1" : Answer1, "answer2" : Answer2, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook,
              "feedback_experiments_input" : Comments,
              "feedback_mentor_support": Mentor_support}
      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:        
        print(r["err"])
        return None   
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://dlfa.iisc.talentsprint.com/notebook_submissions")
        #print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
        return submission_id
    else: submission_id
    

def getAdditional():
  try:
    if not Additional: 
      raise NameError
    else:
      return Additional  
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None
  
def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None
  
  
# def getWalkthrough():
#   try:
#     if not Walkthrough:
#       raise NameError
#     else:
#       return Walkthrough
#   except NameError:
#     print ("Please answer Walkthrough Question")
#     return None
  
def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None
  

def getMentorSupport():
  try:
    if not Mentor_support:
      raise NameError
    else:
      return Mentor_support
  except NameError:
    print ("Please answer Mentor support Question")
    return None

def getAnswer1():
  try:
    if not Answer1:
      raise NameError 
    else: 
      return Answer1
  except NameError:
    print ("Please answer Question 1")
    return None

def getAnswer2():
  try:
    if not Answer2:
      raise NameError 
    else: 
      return Answer2
  except NameError:
    print ("Please answer Question 2")
    return None
  

def getId():
  try: 
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup 
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup() 
else:
  print ("Please complete Id and Password cells before running setup")



### Importing required packages

In [ ]:
import pandas as pd
import numpy as np

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

from keras.models import Model
from keras.layers import Input, Embedding, concatenate, Dense, Bidirectional, Dropout, Flatten, Conv1D, MaxPooling1D

### Loading the dataset

In [ ]:
def load_data(dataset):
  data = pd.read_csv(dataset)
  sentence_data = data.drop('pair_ID', axis=1)
  return sentence_data

In [ ]:
# Load the dataset
sentence_data = load_data('sententence_data.csv')

In [ ]:
# Print the first 5 rows from the data
sentence_data.head()

In [ ]:
# Combining the label columns (entailment_AB, entailment_BA) to create a final labels column
sentence_data['labels'] = sentence_data['entailment_AB'].astype(str) + 'and' + sentence_data['entailment_BA'].astype(str)

# No of unique labels after combining the entailment_AB and entailment_BA
print(len(sentence_data['labels'].unique()))

In [ ]:
# check for the unique label combinations 
print(np.unique(sentence_data['labels']))

In [ ]:
# checking for the output counts 
sentence_data['labels'].value_counts()

### Label Encoding

In [ ]:
# Converting the labels from categorical to numerical
le = LabelEncoder()
sentence_data['labels'] = le.fit_transform(sentence_data['labels'])
sentence_data.head()

### Data pre-processing

In [ ]:
def cleaning_dataset(df):
    
    # Pre-Processing
    # converat all sentences to string format
    df['sentence_A'] = df['sentence_A'].astype(str)
    df['sentence_B'] = df['sentence_B'].astype(str)
    
    # convert all sentences to lower case
    df['sentence_A'] = df['sentence_A'].apply(lambda sentence_A: sentence_A.lower())
    df['sentence_B'] = df['sentence_B'].apply(lambda sentence_B: sentence_B.lower())
       
    return df

In [ ]:
sentence_data = cleaning_dataset(sentence_data)
sentence_data = sentence_data.drop(['entailment_AB','entailment_BA'], axis=1)

In [ ]:
sentence_data.head()

In [ ]:
# Combining both the sentences as a corpus
sentence_data['Sentences'] = sentence_data[['sentence_A', 'sentence_B']].apply(lambda x: str(x[0])+" "+str(x[1]), axis=1)

### Tokenize and Pad sequences

A Neural Network only accepts numeric data, so we need to encode the reviews. Here use keras.Tokenizer() to encode the reviews into integers, where each unique word is automatically indexed (using `fit_on_texts` method) calculates the frequency of each word in our corpus/messages. 

`texts_to_sequences` method finally converts our array of sequences of strings to list of sequences of integers (most frequent word is assigned 1 and so on).

Each reviews has a different length, so we need to add padding (by adding 0) or truncating the words to the same length (in this case, it is the mean of all reviews length) using `keras.preprocessing.sequence.pad_sequences.`

`post`, pad or truncate the words in the back of a sentence
`pre`, pad or truncate the words in front of a sentence

Each word is assigned an integer and that integer is placed in a list. 


For example if we have a sentence “How text to sequence and padding works”. Each word is assigned a number. We suppose how = 1, text = 2, to = 3, sequence = 4, and = 5, padding = 6, works = 7. After texts_to_sequences is called our sentence will look like [1, 2, 3, 4, 5, 6, 7 ]. Now for suppose our MAX_SEQUENCE_LENGTH = 10. After padding our sentence will look like `pre` = [0, 0, 0, 1, 2, 3, 4, 5, 6, 7 ], `post` = [1, 2, 3, 4, 5, 6, 7, 0, 0, 0]

In [ ]:
# Tokenizer class from the keras.preprocessing.text module creates a word-to-index integer dictionary
# Vectorize the text samples
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentence_data['Sentences'])

In [ ]:
sen_seq_A = tokenizer.texts_to_sequences(sentence_data['sentence_A'])
sen_seq_B = tokenizer.texts_to_sequences(sentence_data['sentence_B'])

In [ ]:
max_len = 50

sen_seq_A = pad_sequences(sen_seq_A, maxlen=max_len, padding='post')
sen_seq_B = pad_sequences(sen_seq_B, maxlen=max_len, padding='post')

In [ ]:
print(sen_seq_A)

In [ ]:
print(sen_seq_A.shape, sen_seq_B.shape)

### Load the GloVe word embeddings

**What is GloVe?**

GloVe stands for global vectors for word representation. It is an unsupervised learning algorithm developed by Stanford for generating word embeddings by aggregating global word-word co-occurrence matrix from a corpus. Word embeddings are basically a form of word representation that bridges the human understanding of language to that of a machine. Meaning that two similar words are represented by almost similar vectors that are very closely placed in a vector space. These are essential for solving most Natural language processing problems.The resulting embeddings show interesting linear substructures of the word in vector space.

Thus when using word embeddings, all individual words are represented as real-valued vectors in a predefined vector space. Each word is mapped to one vector and the vector values are learned in a way that resembles a neural network.

Now, let us load the 50-dimensional GloVe embeddings.

In [ ]:
embeddings_index = {}
# Loading the 300-dimensional vector of the model
f = open('/content/glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
# Adding 1 because of reversed 0 index
words_not_found = []
vocab_size = len(tokenizer.word_index) + 1
print('Loaded %s word vectors.' % len(embeddings_index))

embedding_dim = 50

# Create a weight matrix for words in the training data
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i >= vocab_size:
        continue
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
                embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)

In [ ]:
print(tokenizer.word_index)

In [ ]:
print(len(tokenizer.word_index))

### Splitting the data into train and test sets

In [ ]:
X = np.stack((sen_seq_A, sen_seq_B), axis=1)
Y = sentence_data['labels']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, stratify=Y, test_size=0.25, shuffle=True)

In [ ]:
# Check for the shape of train and test sets
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
# Storing the sentence_A and sentence_B data seperately for training
train_s1 = X_train[:,0]
train_s2 = X_train[:,1]
test_s1 = X_test[:,0]
test_s2 = X_test[:,1]

In [ ]:
# Converting labels to array
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
# one-hot encode the labels
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
print(y_train.shape)

### Define the Convolutional Neural Network (CNN) model

<center>
<img src="https://cdn.iisc.talentsprint.com/DLFA/Experiment_related_data/CNN_Sentence.png" width=800px/>
</center>
<br><br>


Deep learning models have achieved remarkable results in  computer  vision and speech recognition in recent years.  Within natural language processing,  much of the work with deep learning methods has involved learning word vector representations through neural language models and performing composition over the learned word vectors for classification .Word vectors, wherein words are projected from a sparse, 1-of-V encoding (here V is the vocabularysize) onto a lower dimensional vector space via a hidden layer, are essentially feature extractors that encode semantic features of words in their dimensions.

Convolutional neural networks (CNN) utilize layers  with  convolving filters that are  applied to local  features.   Originally invented for computer vision, CNN models have subsequently been shown to be effective for NLP and  have  achieved  excellent results in semantic parsing, search query retrieval, sentence  modeling, and other traditional NLP tasks.

In the present work, we train a simple CNN with one layer of convolution on top of word vectors obtained from  an  unsupervised neural language model. These vectors were trained on 100 billion words of Google News, and are publicly available. We initially keep the word vectors static and learn only the other parameters of the model.

From the above figure we can observe that the inputs are words. Each word is represented by a vector of size (50, 100, 200, 300).
Apply different filters on the word vectors to create convolutional feature map. Choose the maximum value of the result from each filter vector for pooled representation and then
apply softmax to perform classification.


We have seen the process by which one feature is extracted  from one filter. The  model uses multiple filters (with varying window sizes) to obtain multiple features. These features form the penultimate layer and are passed to a fully connected and then softmax is used to perform classification.

#### 1-D Convolutions over text

The application of convolutional neural networks is the same as in image data. The only difference is that 1D convolutions are applied instead of 2D convolutions. In images, the kernel slides in 2D but in sequence data like text data the kernel slides in one dimension. 

Let’s now take a look at how this CNN can be built. 

The convolution network will be made of of the following: 

* An embedding layer that turns the data into dense vectors of fixed size. 

* A `Conv1D` with no of filter units and the `relu` activation function.

* A `MaxPooling1D` layer that downsamples the input by taking the maximum value. The pooling operation is used to combine the vectors resulting from different convolution windows into a single $l$-dimensional vector. This is done again by taking the max or the average value observed in resulting vector from the convolutions. Ideally this vector will capture the most relevant features of the sentence/document.

* A `Dense` layer with hidden units for the fully connected layer.
    
* An output layer with the softmax activation function.



**Note:** Refer to the following [link](https://cezannec.github.io/CNN_Text_Classification/) for more details of CNN in Text classification.

In [ ]:
# Define a function which takes the input sequence and generate the vectors using the pretrained word embeddings
# The pass it through the convolutional layers and then the max pooling layer
def cnn_model(input_shape, vocab_size, embeddings, embedding_dim):
  # Input as max sequence length
  model_input = Input(shape=(input_shape,))

  # Embedding layer
  layer = Embedding(vocab_size, 
                 embedding_dim, 
                 weights=[embedding_matrix], 
                 input_length=max_len, 
                 trainable=False)(model_input)
                 
  # Convolutional layer              
  layer = Conv1D(filters=16, kernel_size=3, activation='relu')(layer)
  # MaxPool layer
  layer = MaxPooling1D(pool_size=2)(layer)
  output = Flatten()(layer)

  model = Model(inputs=model_input, outputs=output)
  model.summary()
  return model

#### CNN for Sentence Classification

<center>
<img src="https://cdn.iisc.talentsprint.com/DLFA/Experiment_related_data/CNN_sentence_classification.png" width=900px/>
</center>
<br><br>

Here, we tokenize the sentences (sentence_A and sentence_B) into words and then map the text to sequences and for the input sequences we build the word embedding with the pretrained(word2vec, Glove, FastText) models.

We pass the two sentence(A and B) to the CNN model which shares the same weight parameters, which provides the enocoded representation of the sentences. Then, we concatenate the output sentences(A and B) vectors pass it to the fully connected layers of neural networks to train the model and perform the classification at the final layer.




In [ ]:
# Call the CNN model with max sequence length, embeddings and word embedding dimensions
cnn_model = cnn_model(max_len, vocab_size, embedding_matrix, embedding_dim)

# sentence_A and sentence_B inputs as a max sequence length
input_s1 = Input(shape=(max_len,),dtype='int32')
input_s2 = Input(shape=(max_len,), dtype='int32')

# pass the input sentences to the CNN model which share the same weight parameters
left_out = cnn_model(input_s1)
right_out = cnn_model(input_s2)

In [ ]:
# Concatenate the sentence_A and sentence_B output vectors from the CNN model
merge = concatenate([left_out, right_out], axis=1)

In [ ]:
# Fully connected layers
sumx = Dense(256, activation='relu')(merge)
sumx = Dense(128, activation='relu')(sumx)
sumx = Dense(64, activation='relu')(sumx)
sumx = Dropout(0.5)(sumx)

# Output layer for 7 class classification
pred = Dense(7, activation='softmax')(sumx)

In [ ]:
# Creating the functional model with input sentences and outputs the 7 classes
cnn_model = Model(inputs=[input_s1, input_s2], outputs=pred)
cnn_model.summary()

### Compile and train the model

In [ ]:
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam',\
              metrics=['accuracy'])

# Training the CNN model
history = cnn_model.fit([train_s1,train_s2],
                    y_train, 
                    epochs=10,
                    batch_size=16,
                   validation_data=([test_s1, test_s2], y_test))

In [ ]:
# Get the predictions on the test set
y_pred = cnn_model.predict([test_s1, test_s2])

In [ ]:
yout = np.argmax(y_pred, axis=1)
yout.shape

In [ ]:
y_test = np.argmax(y_test, axis=1)

In [ ]:
yout.shape

In [ ]:
print(y_test.shape, yout.shape)
print(y_test[:5], yout[:5])

### Please answer the questions below to complete the experiment:




In [ ]:
#@title Q.1. What does a 1D convolutional layer do in a CNN for Text Classification?
Answer1 = "" #@param ["","It creates a convolution kernel that is convolved with the input layer over a single spatial dimension to produce a tensor of outputs","It creates a convolution kernel that is convolved with the output layer over a multiple spatial dimension to produce a tensor of outputs","It creates a convolution kernel that is convolved with the output layer over a single spatial dimension to produce a tensor of outputs","It creates a convolution kernel that is convolved with the input layer over a multiple spatial dimension to produce a tensor of outputs"]


In [ ]:
#@title Q.2. Which of the following statements is False?
Answer2 = "" #@param ["","Number of parameters in CNNs are usually less than the number of parameters in Feed forward Neural Networks", "CNNs are prone to overfitting because of less number of parameters", "There are no learnable parameters in Pooling layers", "Pooling layer operates on each feature map independently"]


In [ ]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [ ]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "" #@param {type:"string"}


In [ ]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "" #@param ["","Yes", "No"]


In [ ]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Mentor_support = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id = return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")